# K-operator and PPMI dataset

Using the nilearn library and some of the codes by Sidney Hough, Julian Quevedo, and Pino Cholsaipant for the PPMI dataset, on Parkinson disease.

(https://i.imgur.com/FYaEgKW.jpg)

All the imports and installs we'll need.


In [ ]:
import os
import torch
from torch import nn
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

from nilearn import datasets
from nilearn import plotting
from nilearn.input_data import NiftiMapsMasker
from nilearn.connectome import ConnectivityMeasure

import torch.nn.functional as F
from torch.nn import Sequential, Linear, ReLU, GRU, BatchNorm1d
from torch_geometric.nn import EdgeConv, GCNConv, GraphConv
from torch_geometric.nn import global_mean_pool
from torch_geometric.data import InMemoryDataset, Data, DataLoader
from torch_geometric.utils import from_networkx

import networkx as nx
from networkx.convert_matrix import from_numpy_matrix

In [ ]:
import os
import torch
from torch import nn
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

from nilearn import datasets
from nilearn import plotting
from nilearn.input_data import NiftiMapsMasker
from nilearn.connectome import ConnectivityMeasure

import torch.nn.functional as F
from torch.nn import Sequential, Linear, ReLU, GRU, BatchNorm1d
from torch_geometric.nn import EdgeConv, GCNConv, GraphConv
from torch_geometric.nn import global_mean_pool
from torch_geometric.data import InMemoryDataset, Data, DataLoader
from torch_geometric.utils import from_networkx

import networkx as nx
from networkx.convert_matrix import from_numpy_matrix

In [ ]:
from nilearn import datasets, input_data, connectome
import nibabel as nib
import matplotlib.pyplot as plt
import numpy as np


atlas = datasets.fetch_atlas_msdl()
atlas_filename = atlas.maps
atlas_labels = atlas.labels

import os
import dicom2nifti
import dicom2nifti.settings as settings
import pydicom

dicom_folder = "/Users/mariamannone/Downloads/PPMI_6/100878/rsfMRI_RL/2022-03-29_09_56_04.0/I1571504/"
output_folder = "/Users/mariamannone/Desktop/new_phd_/DICOM_converted_M/"

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Convert DICOM to NIfTI using dicom2nifti
settings.disable_validate_slice_increment()
dicom_files = [os.path.join(dicom_folder, file) for file in os.listdir(dicom_folder) if file.endswith(".dcm")]

# Load the DICOM data
dicom_data = [pydicom.dcmread(file) for file in dicom_files]

# Convert DICOM to NIfTI
nifti_file = os.path.join(output_folder, "output.nii.gz")
dicom2nifti.convert_directory(dicom_folder, nifti_file, compression=True, reorient=True)

# Adapting something using my codes 

In [ ]:
from nilearn import datasets, input_data
import os
import numpy as np
from nilearn.input_data import NiftiMapsMasker



nifti_folder = "/Users/mariamannone/Desktop/new_phd_/DICOM_converted_M/output.nii.gz/"

# Load the MSDL atlas
atlas = datasets.fetch_atlas_msdl()
atlas_filename = atlas.maps

# Create a masker using NiftiMapsMasker
masker = NiftiMapsMasker(maps_img=atlas_filename, standardize=True, memory='nilearn_cache')

# Initialize lists to store time series and labels
time_series = []
labels = []

# Iterate over each NIfTI file in the folder
for file_name in os.listdir(nifti_folder):
    if file_name.endswith('.nii.gz'):
        file_path = os.path.join(nifti_folder, file_name)

        # Load and standardize time series using the masker
        ts = masker.fit_transform(file_path)
        time_series.append(ts)

        # Extract the label from the file name or metadata
        # You may need to adapt this based on your file naming or metadata conventions
        labels.append(file_name.split('_')[1])  # Example: Assumes file_name is like 'subject_label.nii.gz'

# Save time series
for i, ts in enumerate(time_series):
    np.savetxt(f'time_series_{i:04d}.csv', ts, delimiter=',')

Calculating partial correlation matrix for each time series and taking the mean. This will serve as the basis for edges in the graphs we create later.



In [ ]:
corr_measure = ConnectivityMeasure(kind='correlation')
pcorr_measure = ConnectivityMeasure(kind='partial correlation')

corr_matrices = corr_measure.fit_transform(time_series)
pcorr_matrices = pcorr_measure.fit_transform(time_series)

Let's visualize a functional connectome!

In [ ]:

# Define the reordered list of ROI indices
reordered_indices = [8, 9, 15, 10, 16, 29, 22, 23, 31, 6, 5, 28, 0, 1, 3, 4, 25, 26, 12, 27, 30, 11, 14, 18, 17, 37, 38, 2, 7, 19, 20, 21, 13, 24, 32, 33, 35, 34, 36]

# Reorder the labels according to the reordered list of indices
reordered_labels = [atlas_labels[i] for i in reordered_indices]

# Plot the connectivity matrix with reordered labels
plotting.plot_matrix(corr_matrices[0], labels=reordered_labels, colorbar=True, vmax=1, vmin=-1)

# Connectome over brain schematic
coords = atlas.region_coords # Get center coordinates of each ROI
plotting.plot_connectome(corr_matrices[0], coords, edge_threshold="80%", colorbar=True)

In [ ]:
from nilearn import plotting

# Example coordinates from an atlas
coords = atlas.region_coords 

# Define a dictionary for edge properties with transparency
edge_properties = {
    'alpha': 0.3 # 0 is completely transparent, 1 is opaque
}

# Plot the connectome with edge transparency
plotting.plot_connectome(corr_matrices[0], 
                         coords, 
                         edge_threshold="80%", 
                         colorbar=True,
                         edge_kwargs=edge_properties)

# Show the plot
plotting.show()


In [ ]:
# data of a control patient:
dicom_folder_control = "/Users/mariamannone/Downloads/control_test4/101195/rsfMRI_LR/2021-05-17_10_47_24.0/I1496363"

In [ ]:
#dicom_folder_control = "/Users/mariamannone/Downloads/PPMI/100005/rsfMRI_L_R/2021-01-27_14_58_23.0/I1526360/"
output_folder_control = "/Users/mariamannone/Desktop/new_phd_/DICOM_converted_control_/output_control.nii.gz/"


In [ ]:
# CONTROL

import os
import dicom2nifti
import dicom2nifti.settings as settings
import pydicom

dicom_folder = "/Users/mariamannone/Downloads/control_test4/101195/rsfMRI_LR/2021-05-17_10_47_24.0/I1496363"
output_folder = "/Users/mariamannone/Desktop/new_phd_/DICOM_converted_control_/"


# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Convert DICOM to NIfTI using dicom2nifti
settings.disable_validate_slice_increment()
dicom_files = [os.path.join(dicom_folder, file) for file in os.listdir(dicom_folder) if file.endswith(".dcm")]

# Load the DICOM data
dicom_data = [pydicom.dcmread(file) for file in dicom_files]

# Convert DICOM to NIfTI
nifti_file = os.path.join(output_folder, "output_control.nii.gz")
dicom2nifti.convert_directory(dicom_folder, nifti_file, compression=True, reorient=True)

nifti_folder_control = "/Users/mariamannone/Desktop/new_phd_/DICOM_converted_control_/output_control.nii.gz/"

# Load the MSDL atlas
atlas = datasets.fetch_atlas_msdl()
atlas_filename = atlas.maps

# Create a masker using NiftiMapsMasker
masker = NiftiMapsMasker(maps_img=atlas_filename, standardize=True, memory='nilearn_cache')

# Initialize lists to store time series and labels
time_series_control = []
labels = []

# Iterate over each NIfTI file in the folder
for file_name in os.listdir(nifti_folder_control):
    if file_name.endswith('.nii.gz'):
        file_path = os.path.join(nifti_folder_control, file_name)

        # Load and standardize time series using the masker
        ts = masker.fit_transform(file_path)
        time_series_control.append(ts)

        # Extract the label from the file name or metadata
        # You may need to adapt this based on your file naming or metadata conventions
        labels.append(file_name.split('_')[1])  # Example: Assumes file_name is like 'subject_label.nii.gz'

# Save time series
for i, ts in enumerate(time_series):
    np.savetxt(f'time_series_control{i:04d}.csv', ts, delimiter=',')

In [ ]:
corr_measure_control = ConnectivityMeasure(kind='correlation')
pcorr_measure_control = ConnectivityMeasure(kind='partial correlation')

corr_matrices_control = corr_measure.fit_transform(time_series_control)
pcorr_matrices_control = pcorr_measure.fit_transform(time_series_control)

In [ ]:
# Partial correlation matrix

reordered_indices = [8, 9, 15, 10, 16, 29, 22, 23, 31, 6, 5, 28, 0, 1, 3, 4, 25, 26, 12, 27, 30, 11, 14, 18, 17, 37, 38, 2, 7, 19, 20, 21, 13, 24, 32, 33, 35, 34, 36]

# Reorder the labels according to the reordered list of indices
reordered_labels = [atlas_labels[i] for i in reordered_indices]

# Plot the connectivity matrix with reordered labels
plotting.plot_matrix(corr_matrices_control[0], labels=reordered_labels, colorbar=True, vmax=1, vmin=-1)


# Connectome over brain schematic
coords = atlas.region_coords # Get center coordinates of each ROI
plotting.plot_connectome(corr_matrices_control[0], coords, edge_threshold="80%", colorbar=True)

In [ ]:
# Define a dictionary for edge properties with transparency
edge_properties = {
    'alpha': 0.3 # 0 is completely transparent, 1 is opaque
}

# Plot the connectome with edge transparency
plotting.plot_connectome(corr_matrices_control[0], 
                         coords, 
                         edge_threshold="80%", 
                         colorbar=True,
                         edge_kwargs=edge_properties)

# Show the plot
plotting.show()


In [ ]:

#####

# Inverse matrix of the "gesund Gehirn"

# Matrix multiplication between the "krank Gehirn" and the inverse gesund Gehirn
# This should yield an approximation of the K-operator in matrix terms


import numpy as np


# Load matrix computed above for the control patient
matrix1 = corr_matrices_control[0]

# Load matrix computed above for the PD patient
matrix2 = corr_matrices[0]

# Compute the inverse of the matrix
inverse_matrix1 = np.linalg.inv(matrix1)

K_approximation = np.dot(matrix2,inverse_matrix1)

vmin = -6
vmax = 6

# matrix 1
plotting.plot_matrix(corr_matrices_control[0], labels = atlas_labels, colorbar=True, vmax=1, vmin=-1)
coords = atlas.region_coords
plotting.plot_connectome(corr_matrices_control[0], coords, edge_threshold = "80%", colorbar=True, edge_vmin=-1, edge_vmax=1)

# matrix 2
plotting.plot_matrix(corr_matrices[0], labels = atlas_labels, colorbar=True, vmax=1, vmin=-1)
coords = atlas.region_coords
plotting.plot_connectome(corr_matrices[0], coords, edge_threshold = "80%", colorbar=True, edge_vmin=-1, edge_vmax=1)

# Define the reordered list of ROI indices
reordered_indices = [8, 9, 15, 10, 16, 29, 22, 23, 31, 6, 5, 28, 0, 1, 3, 4, 25, 26, 12, 27, 30, 11, 14, 18, 17, 37, 38, 2, 7, 19, 20, 21, 13, 24, 32, 33, 35, 34, 36]

# Reorder the labels according to the reordered list of indices
reordered_labels = [atlas_labels[i] for i in reordered_indices]

# K-operator
plotting.plot_matrix(K_approximation, labels = reordered_labels, colorbar=True, vmax=3, vmin=-3) # vmax=1, vmin=-0.8
coords = atlas.region_coords
plotting.plot_connectome(K_approximation, coords, edge_threshold = "80%", colorbar=True, edge_vmin=vmin, edge_vmax=vmax)

# Check if matrix1_approx is close to matrix2
tolerance = 1e-6
is_close = np.allclose(matrix2, np.dot(K_approximation,matrix1), atol=tolerance)

if is_close:
    print("KG is close to G^p.")
else:
    print("KG is not close to G^p.")
    
    
from sympy import Matrix, latex


In [ ]:
# MARCH 20

# PROXIMITY OF THE ROIs ACCORDING TO THEIR BELONGING TO THE SAME LOBE

# SHOWING A SQUARE TO DELIMIT THE ROIs INSIDE THE SAME LOBE

# WHY 6 blocks if the lobes are 5 in the following representations??
import numpy as np
from nilearn import plotting, datasets
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

# Load the MSDL atlas for labels and coordinates
atlas = datasets.fetch_atlas_msdl()
atlas_labels = atlas.labels
coords = atlas.region_coords

lobe_mapping = {
    'Frontal': [8, 9, 15, 10, 16, 29, 22, 23, 31, 6, 5, 28],
    'Temporal': [0, 1, 3, 4, 25, 26, 12],
    'Parietal': [27, 30, 11, 14, 18, 17, 37, 38],
    'Occipital': [2, 7, 19, 20, 21],
    'Subcortical': [13, 24, 32, 33, 35, 34, 36]
}

# Reorder atlas labels and coordinates based on lobes
reordered_labels = []
reordered_coords = []
lobe_rectangles = {}
for lobe, indices in lobe_mapping.items():
    lobe_labels = [atlas_labels[i] for i in indices]
    lobe_coords = [coords[i] for i in indices]
    reordered_labels.extend(lobe_labels)
    reordered_coords.extend(lobe_coords)
    
    # Calculate the bounding box for the lobe
    lobe_min_x = min(coord[0] for coord in lobe_coords)
    lobe_max_x = max(coord[0] for coord in lobe_coords)
    lobe_min_y = min(coord[1] for coord in lobe_coords)
    lobe_max_y = max(coord[1] for coord in lobe_coords)
    lobe_rectangles[lobe] = ((lobe_min_x, lobe_min_y), lobe_max_x - lobe_min_x, lobe_max_y - lobe_min_y)

# Assuming you have matrices defined: corr_matrices_control, corr_matrices
matrix1 = corr_matrices_control[0]
matrix2 = corr_matrices[0]
inverse_matrix1 = np.linalg.inv(matrix1)
K_approximation = np.dot(matrix2, inverse_matrix1)

# Plotting
vmin = -6
vmax = 6

# Plot matrix 1
fig, ax = plt.subplots()
plotting.plot_matrix(matrix1, labels=reordered_labels, colorbar=True, vmax=1, vmin=-1, axes=ax)

# Add rectangles for lobes
for lobe, (corner, width, height) in lobe_rectangles.items():
    rect = Rectangle(corner, width, height, linewidth=2, edgecolor='r', facecolor='none')
    ax.add_patch(rect)

# Plot matrix 2
fig, ax = plt.subplots()
plotting.plot_matrix(matrix2, labels=reordered_labels, colorbar=True, vmax=1, vmin=-1, axes=ax)

# Add rectangles for lobes
for lobe, (corner, width, height) in lobe_rectangles.items():
    rect = Rectangle(corner, width, height, linewidth=2, edgecolor='r', facecolor='none')
    ax.add_patch(rect)

# Plot K-operator
fig, ax = plt.subplots()
plotting.plot_matrix(K_approximation, labels=reordered_labels, colorbar=True, vmax=6, vmin=-6, axes=ax)

# Add rectangles for lobes
for lobe, (corner, width, height) in lobe_rectangles.items():
    rect = Rectangle(corner, width, height, linewidth=2, edgecolor='r', facecolor='none')
    ax.add_patch(rect)

plt.show()


In [ ]:
from nilearn import datasets

# Load the MSDL atlas for labels
atlas = datasets.fetch_atlas_msdl()
atlas_labels = atlas.labels

# Define a custom mapping to group ROIs by brain lobes
lobe_mapping = {
    'Frontal': [8, 9, 15, 10, 16, 29, 22, 23, 31, 6, 5, 28],
    'Temporal': [0, 1, 3, 4, 25, 26, 12],
    'Parietal': [27, 30, 11, 14, 18, 17, 37, 38],
    'Occipital': [2, 7, 19, 20, 21],
    'Subcortical': [13, 24, 32, 33, 35, 34, 36]
}

# Print correspondence between matrix blocks and lobes
print("\nCorrespondence between matrix blocks and lobes:")
for lobe, indices in lobe_mapping.items():
    print(f"{lobe}:")
    for idx in indices:
        print(f"    Block {idx + 1}: {atlas_labels[idx]}")


In [ ]:
# SAVE the information here:


gesund_alt = corr_matrices_control[0]

krank_alt = corr_matrices[0]

K_male_alt = K_approximation


In [ ]:
# Addendum of March 26


import numpy as np

# Define the sizes of the submatrices
sizes = [14, 6, 10, 6, 3]

# Generate some example submatrices
submatrices = [np.random.rand(size, size) for size in sizes]

# Initialize the block-diagonal matrix
block_diagonal_matrix = np.zeros((sum(sizes), sum(sizes)))

# Fill the block-diagonal matrix with the submatrices
row_idx = 0
for size, submatrix in zip(sizes, submatrices):
    block_diagonal_matrix[row_idx:row_idx+size, row_idx:row_idx+size] = submatrix
    row_idx += size

# Print the block-diagonal matrix
print("Block-Diagonal Matrix:")
print(block_diagonal_matrix)

In [ ]:
# PRINT the K-operator as tensor product of matrices acting on each region of interest:

In [ ]:
# This is important:
tensor_product_all_submatrices = np.eye(len(atlas_labels))

#   return np.kron(submatrix, matrix2 @ inverse_matrix1)

In [ ]:
# Check and print shapes of matrices
print("Shape of matrix2 @ inverse_matrix1 :", (matrix2 @ inverse_matrix1).shape)
print("Shape of tensor_product_all_submatrices:", tensor_product_all_submatrices.shape)

# Perform matrix multiplication
# result_matrix = matrix1 @ tensor_product_all_submatrices
result_matrix = (matrix2 @ inverse_matrix1) @ tensor_product_all_submatrices
print("Shape of the reconstructed matrix:", result_matrix.shape)


In [ ]:
import numpy as np
from nilearn import plotting, datasets

# Load the MSDL atlas for labels and coordinates
atlas = datasets.fetch_atlas_msdl()
atlas_filename = atlas.maps
atlas_labels = atlas.labels
coords = atlas.region_coords

# Load matrix computed above for the control patient
matrix1 = corr_matrices_control[0]

# Load matrix computed above for the PD patient
matrix2 = corr_matrices[0]

# Compute the inverse of the matrix
inverse_matrix1 = np.linalg.inv(matrix1)

# Compute the K_approximation matrix
K_approximation = np.dot(matrix2, inverse_matrix1)

# Function to create tensor product submatrices
def create_tensor_product_submatrix(n, i, j):
    submatrix = np.zeros((n, n))
    submatrix[i, j] = 1  # Set the (i, j)-th element to 1, rest are zeros
    #plotting.plot_matrix(submatrix, colorbar=True, vmax=1, vmin=-0.8, title=title)
    return np.kron(submatrix, matrix2 @ inverse_matrix1)

# Check tensor product submatrices
n = len(atlas_labels)
for i in range(n):
    for j in range(n):
        tensor_product_submatrix = create_tensor_product_submatrix(n, i, j)
    
        # Print or visualize the tensor product submatrix as needed
        print(f"Tensor Product Submatrix for ROI {atlas_labels[i]} and {atlas_labels[j]}:")
        print(tensor_product_submatrix)
        print("\n")
        # Plot heatmap for the submatrix
        title = f"Heatmap for Submatrix {atlas_labels[i]}-{atlas_labels[j]}"
        plotting.plot_matrix(tensor_product_submatrix, colorbar=True, vmax=1, vmin=-0.8, title=title)
        #plotting.plot_matrix(roi_submatrix, colorbar=True, vmax=1, vmin=-0.8, title=title)
        plotting.show()


if matrix1.shape[1] != tensor_product_all_submatrices.shape[0]:
    raise ValueError("Incompatible matrix dimensions for multiplication.")

# Perform matrix multiplication
# result_matrix = matrix1 @ tensor_product_all_submatrices
result_matrix = (matrix2 @ inverse_matrix1) @ tensor_product_all_submatrices


# Now you can use result_matrix for further analysis or checks
print("Result matrix shape:", result_matrix.shape)

# Assuming you have matrices defined: K_approximation and result_matrix

# Scale or normalize matrices (optional step)
scaled_result_matrix = result_matrix / np.linalg.norm(result_matrix)
scaled_K_approximation = K_approximation / np.linalg.norm(K_approximation)

# Check if the result is close to the original matrix
tolerance = 0.0001 # 1e-6
is_close = np.allclose(scaled_result_matrix, scaled_K_approximation, atol=tolerance)

if is_close:
    print("Result matrix is close to the original K_approximation.")
else:
    print("Result matrix is not close to the original K_approximation.")

# Visualize matrices (assuming you have matrices defined: K_approximation, result_matrix, matrix1, matrix2)
plotting.plot_matrix(K_approximation, labels=atlas_labels, colorbar=True, vmax=1, vmin=-0.8, title="Original K_approximation")
plotting.plot_matrix(result_matrix, labels=atlas_labels, colorbar=True, vmax=1, vmin=-0.8, title="Reconstructed Matrix")

# Show the plots
plotting.show()


In [ ]:
import numpy as np
from nilearn import plotting, datasets

# Load the MSDL atlas for labels and coordinates
atlas = datasets.fetch_atlas_msdl()
atlas_filename = atlas.maps
atlas_labels = atlas.labels
coords = atlas.region_coords

# Load matrix computed above for the control patient
matrix1 = corr_matrices_control[0]

# Load matrix computed above for the PD patient
matrix2 = corr_matrices[0]

# Compute the inverse of the matrix
inverse_matrix1 = np.linalg.inv(matrix1)

# Function to create tensor product submatrices and plot heatmaps
def visualize_submatrix(i, j):
    submatrix = np.zeros((1, 1))
    submatrix[0, 0] = matrix2[i, j] * inverse_matrix1[j, i]  # Set the (0, 0)-th element to the corresponding value
    
    # Plot heatmap for the submatrix
    title = f"Heatmap for Submatrix {atlas_labels[i]}-{atlas_labels[j]}"
    plotting.plot_matrix(submatrix, colorbar=True, vmax=1, vmin=-0.8, title=title)
    plotting.show()

# Check tensor product submatrices and plot heatmaps
n = len(atlas_labels)
for i in range(n):
    for j in range(n):
        visualize_submatrix(i, j)


In [ ]:
# MARCH 12


# Blocks: submatrices corresponding to the lobes

import numpy as np
from nilearn import plotting, datasets

# Load the MSDL atlas for labels and coordinates
atlas = datasets.fetch_atlas_msdl()
atlas_labels = atlas.labels
coords = atlas.region_coords

lobe_mapping = {
    'Frontal': [8, 9, 15, 10, 16, 29, 22, 23, 31, 6, 5, 28],
    'Temporal': [0, 1, 3, 4, 25, 26, 12],
    'Parietal': [27, 30, 11, 14, 18, 17, 37, 38],
    'Occipital': [2, 7, 19, 20, 21],
    'Subcortical etc': [13, 24, 32, 33, 35, 34, 36]
}

# Load matrices computed for the control and PD patients
matrix1 = corr_matrices_control[0]
matrix2 = corr_matrices[0]

# Compute the inverse of the matrix1
inverse_matrix1 = np.linalg.inv(matrix1)

# Function to visualize submatrix for a given lobe
def visualize_submatrix_for_lobe(lobe):
    lobe_indices = lobe_mapping[lobe]
    submatrix = np.zeros((len(lobe_indices), len(lobe_indices)))
    submatrix_labels = [atlas_labels[i] for i in lobe_indices]  # Get ROI labels
    
    for i, roi1 in enumerate(lobe_indices):
        for j, roi2 in enumerate(lobe_indices):
            submatrix[i, j] = matrix2[roi1, roi2] * inverse_matrix1[roi2, roi1]
    
    # Plot heatmap for the submatrix
    title = f"Heatmap for Submatrix of {lobe}"
    plotting.plot_matrix(submatrix, labels=submatrix_labels, colorbar=True, vmax=1, vmin=-1, title=title)
    plotting.show()

# Visualize submatrices for each lobe
for lobe in lobe_mapping:
    visualize_submatrix_for_lobe(lobe)




In [ ]:

# Addendum of March 26

import numpy as np
from nilearn import plotting, datasets

# Load the MSDL atlas for labels and coordinates
atlas = datasets.fetch_atlas_msdl()
atlas_labels = atlas.labels
coords = atlas.region_coords

# Define a custom mapping to group ROIs by brain lobes
lobe_mapping = {
    'Frontal': [8, 9, 15, 10, 16, 29, 22, 23, 31, 6, 5, 28],
    'Temporal': [0, 1, 3, 4, 25, 26, 12],
    'Parietal': [27, 30, 11, 14, 18, 17, 37, 38],
    'Occipital': [2, 7, 19, 20, 21],
    'Subcortical': [13, 24, 32, 33, 35, 34, 36]
}

# Load matrices computed for the control and PD patients
# Replace these with your actual data
matrix1 = corr_matrices_control[0]
matrix2 = corr_matrices[0]

# Function to create block-diagonal matrix from submatrices
def create_block_diagonal_matrix(submatrices, lobe_mapping):
    # Calculate the total number of ROIs
    num_rois = sum(len(roi_list) for roi_list in lobe_mapping.values())
    
    # Initialize the block-diagonal matrix
    block_diagonal_matrix = np.zeros((num_rois, num_rois))
    
    # Fill the block-diagonal matrix with the submatrices
    row_idx = 0
    for lobe, roi_list in lobe_mapping.items():
        size = len(roi_list)
        block_diagonal_matrix[row_idx:row_idx+size, row_idx:row_idx+size] = submatrices[lobe]
        row_idx += size
    
    return block_diagonal_matrix

# Compute the inverse of the matrix1
inverse_matrix1 = np.linalg.inv(matrix1)

# Compute the submatrices for each lobe
submatrices = {}
for lobe, roi_list in lobe_mapping.items():
    size = len(roi_list)
    submatrix = np.zeros((size, size))
    for i, roi1 in enumerate(roi_list):
        for j, roi2 in enumerate(roi_list):
            submatrix[i, j] = matrix2[roi1, roi2] * inverse_matrix1[roi2, roi1]
    submatrices[lobe] = submatrix

# Create the block-diagonal matrix
block_diagonal_matrix = create_block_diagonal_matrix(submatrices, lobe_mapping)


# Define the reordered list of ROI indices
reordered_indices = [8, 9, 15, 10, 16, 29, 22, 23, 31, 6, 5, 28, 0, 1, 3, 4, 25, 26, 12, 27, 30, 11, 14, 18, 17, 37, 38, 2, 7, 19, 20, 21, 13, 24, 32, 33, 35, 34, 36]

# Reorder the labels according to the reordered list of indices
reordered_labels = [atlas_labels[i] for i in reordered_indices]

# Plot heatmap for the block-diagonal matrix
title = "K-operator as a block-diagonal matrix"
plotting.plot_matrix(block_diagonal_matrix, labels=reordered_labels, colorbar=True, vmax=1, vmin=-1, title=title)


print("Size of the block-diagonal matrix:", block_diagonal_matrix.shape)

In [ ]:
# MARCH 18

# Inverse of K


# Define the reordered list of ROI indices
reordered_indices = [8, 9, 15, 10, 16, 29, 22, 23, 31, 6, 5, 28, 0, 1, 3, 4, 25, 26, 12, 27, 30, 11, 14, 18, 17, 37, 38, 2, 7, 19, 20, 21, 13, 24, 32, 33, 35, 34, 36]

# Reorder the labels according to the reordered list of indices
reordered_labels = [atlas_labels[i] for i in reordered_indices]

K_inv = np.linalg.inv(K_approximation)
plotting.plot_matrix(K_inv, labels = reordered_labels, colorbar=True, vmax=6, vmin=-6) # vmax=1, vmin=-0.8
coords = atlas.region_coords
plotting.plot_connectome(K_inv, coords, edge_threshold = "80%", colorbar=True, edge_vmin=vmin, edge_vmax=vmax)


# Load the MSDL atlas for labels and coordinates
atlas = datasets.fetch_atlas_msdl()
atlas_labels = atlas.labels
coords = atlas.region_coords

lobe_mapping = {
    'Frontal': [8, 9, 15, 10, 16, 29, 22, 23, 31, 6, 5, 28],
    'Temporal': [0, 1, 3, 4, 25, 26, 12],
    'Parietal': [27, 30, 11, 14, 18, 17, 37, 38],
    'Occipital': [2, 7, 19, 20, 21],
    'Subcortical': [13, 24, 32, 33, 35, 34, 36]
}

# Function to create block-diagonal matrix from submatrices
def create_block_diagonal_matrix(submatrices, lobe_mapping):
    # Calculate the total number of ROIs
    num_rois = sum(len(roi_list) for roi_list in lobe_mapping.values())
    
    # Initialize the block-diagonal matrix
    block_diagonal_matrix = np.zeros((num_rois, num_rois))
    
    # Fill the block-diagonal matrix with the submatrices
    row_idx = 0
    for lobe, roi_list in lobe_mapping.items():
        size = len(roi_list)
        block_diagonal_matrix[row_idx:row_idx+size, row_idx:row_idx+size] = submatrices[lobe]
        row_idx += size
    
    return block_diagonal_matrix

# Compute the inverse of the matrix1
inverse_matrix1 = np.linalg.inv(matrix1)

# Compute the submatrices for each lobe
submatrices = {}
for lobe, roi_list in lobe_mapping.items():
    size = len(roi_list)
    submatrix = np.zeros((size, size))
    for i, roi1 in enumerate(roi_list):
        for j, roi2 in enumerate(roi_list):
            submatrix[i, j] = K_inv[roi1, roi2]
    submatrices[lobe] = submatrix

# Create the block-diagonal matrix
block_diagonal_matrix = create_block_diagonal_matrix(submatrices, lobe_mapping)

# Plot heatmap for the block-diagonal matrix
title = "Inverse K-operator as a block-diagonal matrix"
plotting.plot_matrix(block_diagonal_matrix, labels = reordered_labels, colorbar=True, vmax=6, vmin=-6, title=title)


print("Size of the block-diagonal matrix:", block_diagonal_matrix.shape)


In [ ]:
from nilearn import datasets

# Load the MSDL atlas for labels and coordinates
atlas = datasets.fetch_atlas_msdl()
atlas_labels = atlas.labels
atlas_descriptions = atlas.description

# Print the list of ROIs with their descriptions
for index, (label, description) in enumerate(zip(atlas_labels, atlas_descriptions)):
    print(f"ROI {index}: {label} - {description}")


In [ ]:
from nilearn import datasets, input_data, connectome
import nibabel as nib
import matplotlib.pyplot as plt
import numpy as np


# Load the MSDL atlas DIRECTLY HERE
atlas = datasets.fetch_atlas_msdl()
atlas_filename = atlas.maps

import os
import dicom2nifti
import dicom2nifti.settings as settings
import pydicom


dicom_folder = "/Users/mariamannone/Downloads/time_points_PPMI/100006/rsfMRI_RL/2020-12-16_13_26_04.0/I1526375"
output_folder = "/Users/mariamannone/Desktop/new_phd_/DICOM_converted_Female_baseline/"

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Convert DICOM to NIfTI using dicom2nifti
settings.disable_validate_slice_increment()
dicom_files = [os.path.join(dicom_folder, file) for file in os.listdir(dicom_folder) if file.endswith(".dcm")]

# Load the DICOM data
dicom_data = [pydicom.dcmread(file) for file in dicom_files]

# Convert DICOM to NIfTI
nifti_file = os.path.join(output_folder, "output.nii.gz")
dicom2nifti.convert_directory(dicom_folder, nifti_file, compression=True, reorient=True)

In [ ]:

from nilearn import datasets, input_data
import os
import numpy as np
from nilearn.input_data import NiftiMapsMasker



nifti_folder = "/Users/mariamannone/Desktop/new_phd_/DICOM_converted_Female_baseline/output.nii.gz/"

# Load the MSDL atlas
atlas = datasets.fetch_atlas_msdl()
atlas_filename = atlas.maps

# Create a masker using NiftiMapsMasker
masker = NiftiMapsMasker(maps_img=atlas_filename, standardize=True, memory='nilearn_cache')

# Initialize lists to store time series and labels
time_series = []
labels = []

# Iterate over each NIfTI file in the folder
for file_name in os.listdir(nifti_folder):
    if file_name.endswith('.nii.gz'):
        file_path = os.path.join(nifti_folder, file_name)

        # Load and standardize time series using the masker
        ts = masker.fit_transform(file_path)
        time_series.append(ts)

        # Extract the label from the file name or metadata
        # You may need to adapt this based on your file naming or metadata conventions
        labels.append(file_name.split('_')[1])  # Example: Assumes file_name is like 'subject_label.nii.gz'

# Save time series
for i, ts in enumerate(time_series):
    np.savetxt(f'time_series_{i:04d}.csv', ts, delimiter=',')




In [ ]:


corr_measure = ConnectivityMeasure(kind='correlation')
pcorr_measure = ConnectivityMeasure(kind='partial correlation')

corr_matrices = corr_measure.fit_transform(time_series)
pcorr_matrices = pcorr_measure.fit_transform(time_series)



# Partial correlation matrix
plotting.plot_matrix(corr_matrices[0], labels=reordered_labels, colorbar=True, vmax=1, vmin=-0.8)

# Connectome over brain schematic
coords = atlas.region_coords # Get center coordinates of each ROI
plotting.plot_connectome(corr_matrices[0], coords, edge_threshold="80%", colorbar=True)



In [ ]:
# Define a dictionary for edge properties with transparency
edge_properties = {
    'alpha': 0.3 # 0 is completely transparent, 1 is opaque
}

# Plot the connectome with edge transparency
plotting.plot_connectome(corr_matrices[0], 
                         coords, 
                         edge_threshold="80%", 
                         colorbar=True,
                         edge_kwargs=edge_properties)

# Show the plot
plotting.show()

In [ ]:


# FIRST FOLLOW-UP
# Female_fu



import os
import dicom2nifti
import dicom2nifti.settings as settings
import pydicom

dicom_folder = "/Users/mariamannone/Downloads/time_points_PPMI/100006/rsfMRI_RL/2021-12-15_15_16_17.0/I1530145/"
output_folder = "/Users/mariamannone/Desktop/new_phd_/DICOM_converted_Female_fu/"



# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Convert DICOM to NIfTI using dicom2nifti
settings.disable_validate_slice_increment()
dicom_files = [os.path.join(dicom_folder, file) for file in os.listdir(dicom_folder) if file.endswith(".dcm")]

# Load the DICOM data
dicom_data = [pydicom.dcmread(file) for file in dicom_files]

# Convert DICOM to NIfTI
nifti_file = os.path.join(output_folder, "output_fu.nii.gz")
dicom2nifti.convert_directory(dicom_folder, nifti_file, compression=True, reorient=True)




nifti_folder_fu = "/Users/mariamannone/Desktop/new_phd_/DICOM_converted_Female_fu/output_fu.nii.gz/"



# Load the MSDL atlas
atlas = datasets.fetch_atlas_msdl()
atlas_filename = atlas.maps

# Create a masker using NiftiMapsMasker
masker = NiftiMapsMasker(maps_img=atlas_filename, standardize=True, memory='nilearn_cache')

# Initialize lists to store time series and labels
time_series_fu = []
labels = []

# Iterate over each NIfTI file in the folder
for file_name in os.listdir(nifti_folder_fu):
    if file_name.endswith('.nii.gz'):
        file_path = os.path.join(nifti_folder_fu, file_name)

        # Load and standardize time series using the masker
        ts = masker.fit_transform(file_path)
        time_series_fu.append(ts)

        # Extract the label from the file name or metadata
        # You may need to adapt this based on your file naming or metadata conventions
        labels.append(file_name.split('_')[1])  # Example: Assumes file_name is like 'subject_label.nii.gz'

# Save time series
for i, ts in enumerate(time_series_fu):
    np.savetxt(f'time_series_fu{i:04d}.csv', ts, delimiter=',')

In [ ]:
from nilearn import datasets, input_data
import os
import numpy as np
from nilearn.input_data import NiftiMapsMasker



nifti_folder = "/Users/mariamannone/Desktop/new_phd_/DICOM_converted_Female_fu/output_fu.nii.gz/"

# Load the MSDL atlas
atlas = datasets.fetch_atlas_msdl()
atlas_filename = atlas.maps

# Create a masker using NiftiMapsMasker
masker = NiftiMapsMasker(maps_img=atlas_filename, standardize=True, memory='nilearn_cache')

# Initialize lists to store time series and labels
time_series_fu = []
labels = []

# Iterate over each NIfTI file in the folder
for file_name in os.listdir(nifti_folder):
    if file_name.endswith('.nii.gz'):
        file_path = os.path.join(nifti_folder, file_name)

        # Load and standardize time series using the masker
        ts = masker.fit_transform(file_path)
        time_series_fu.append(ts)

        # Extract the label from the file name or metadata
        # You may need to adapt this based on your file naming or metadata conventions
        labels.append(file_name.split('_')[1])  # Example: Assumes file_name is like 'subject_label.nii.gz'

# Save time series
for i, ts in enumerate(time_series_fu):
    np.savetxt(f'time_series_fu{i:04d}.csv', ts, delimiter=',')





In [ ]:
corr_measure_fu = ConnectivityMeasure(kind='correlation')
pcorr_measure_fu = ConnectivityMeasure(kind='partial correlation')

corr_matrices_fu = corr_measure.fit_transform(time_series_fu)
pcorr_matrices_fu = pcorr_measure.fit_transform(time_series_fu)


In [ ]:

# Partial correlation matrix
plotting.plot_matrix(corr_matrices_fu[0], labels=reordered_labels, colorbar=True, vmax=1, vmin=-0.8)

# Connectome over brain schematic
coords = atlas.region_coords # Get center coordinates of each ROI
plotting.plot_connectome(corr_matrices_fu[0], coords, edge_threshold="80%", colorbar=True)




In [ ]:
# Define a dictionary for edge properties with transparency
edge_properties = {
    'alpha': 0.3 # 0 is completely transparent, 1 is opaque
}

# Plot the connectome with edge transparency
plotting.plot_connectome(corr_matrices_fu[0], 
                         coords, 
                         edge_threshold="80%", 
                         colorbar=True,
                         edge_kwargs=edge_properties)

# Show the plot
plotting.show()


In [ ]:

#####

# Inverse matrix of the "time zero"


import numpy as np


# Load matrix computed above for the BASELINE
matrix1 = corr_matrices[0]

# Load matrix computed above for FIRST FOLLOW-UP
matrix2 = corr_matrices_fu[0]

# Compute the inverse of the matrix
inverse_matrix1 = np.linalg.inv(matrix1)

K_approximation = np.dot(matrix2,inverse_matrix1)

# print(K_approximation)


# Plots

vmin = -1
vmax = 1

# matrix 1
plotting.plot_matrix(corr_matrices_fu[0], labels = atlas_labels, colorbar=True, vmax=1, vmin=-1)
coords = atlas.region_coords
plotting.plot_connectome(corr_matrices_fu[0], coords, edge_threshold = "80%", colorbar=True, edge_vmin=vmin, edge_vmax=vmax)

# matrix 2
plotting.plot_matrix(corr_matrices[0], labels = atlas_labels, colorbar=True, vmax=1, vmin=-1)
coords = atlas.region_coords
plotting.plot_connectome(corr_matrices[0], coords, edge_threshold = "80%", colorbar=True, edge_vmin=vmin, edge_vmax=vmax)

# K-operator
plotting.plot_matrix(K_approximation, labels = atlas_labels, colorbar=True, vmax=10, vmin=-10) # vmax=1, vmin=-0.8
coords = atlas.region_coords
plotting.plot_connectome(K_approximation, coords, edge_threshold = "80%", colorbar=True, edge_vmin=vmin, edge_vmax=vmax)



# Check if matrix1_approx is close to matrix2
tolerance = 1e-6
is_close = np.allclose(matrix2, np.dot(K_approximation,matrix1), atol=tolerance)

if is_close:
    print("KG is close to G^p.")
else:
    print("KG is not close to G^p.")
    
    
from sympy import Matrix, latex


In [ ]:

# Addendum of March 26:


import numpy as np
from nilearn import plotting, datasets

# Load the MSDL atlas for labels and coordinates
atlas = datasets.fetch_atlas_msdl()
atlas_labels = atlas.labels
coords = atlas.region_coords

# Define a custom mapping to group ROIs by brain lobes
lobe_mapping = {
    'Frontal': [8, 9, 15, 10, 16, 29, 22, 23, 31, 6, 5, 28],
    'Temporal': [0, 1, 3, 4, 25, 26, 12],
    'Parietal': [27, 30, 11, 14, 18, 17, 37, 38],
    'Occipital': [2, 7, 19, 20, 21],
    'Subcortical': [13, 24, 32, 33, 35, 34, 36]
}

# Load matrices computed for the control and PD patients
matrix1 = corr_matrices[0]
matrix2 = corr_matrices_fu[0]

# Function to create block-diagonal matrix from submatrices
def create_block_diagonal_matrix(submatrices, lobe_mapping):
    # Calculate the total number of ROIs
    num_rois = sum(len(roi_list) for roi_list in lobe_mapping.values())
    
    # Initialize the block-diagonal matrix
    block_diagonal_matrix = np.zeros((num_rois, num_rois))
    
    # Fill the block-diagonal matrix with the submatrices
    row_idx = 0
    for lobe, roi_list in lobe_mapping.items():
        size = len(roi_list)
        block_diagonal_matrix[row_idx:row_idx+size, row_idx:row_idx+size] = submatrices[lobe]
        row_idx += size
    
    return block_diagonal_matrix

# Compute the inverse of the matrix1
inverse_matrix1 = np.linalg.inv(matrix1)

# Compute the submatrices for each lobe
submatrices = {}
for lobe, roi_list in lobe_mapping.items():
    size = len(roi_list)
    submatrix = np.zeros((size, size))
    for i, roi1 in enumerate(roi_list):
        for j, roi2 in enumerate(roi_list):
            submatrix[i, j] = matrix2[roi1, roi2] * inverse_matrix1[roi2, roi1]
    submatrices[lobe] = submatrix

# Create the block-diagonal matrix
block_diagonal_matrix = create_block_diagonal_matrix(submatrices, lobe_mapping)


# Define the reordered list of ROI indices
reordered_indices = [8, 9, 15, 10, 16, 29, 22, 23, 31, 6, 5, 28, 0, 1, 3, 4, 25, 26, 12, 27, 30, 11, 14, 18, 17, 37, 38, 2, 7, 19, 20, 21, 13, 24, 32, 33, 35, 34, 36]

# Reorder the labels according to the reordered list of indices
reordered_labels = [atlas_labels[i] for i in reordered_indices]


# Plot heatmap for the block-diagonal matrix
title = "K-operator as a block-diagonal matrix"
plotting.plot_matrix(block_diagonal_matrix, labels = reordered_labels, colorbar=True, vmax=1, vmin=-1, title=title)


print("Size of the block-diagonal matrix:", block_diagonal_matrix.shape)


In [ ]:
# MARCH 18

# Inverse of K


# Define the reordered list of ROI indices
reordered_indices = [8, 9, 15, 10, 16, 29, 22, 23, 31, 6, 5, 28, 0, 1, 3, 4, 25, 26, 12, 27, 30, 11, 14, 18, 17, 37, 38, 2, 7, 19, 20, 21, 13, 24, 32, 33, 35, 34, 36]

# Reorder the labels according to the reordered list of indices
reordered_labels = [atlas_labels[i] for i in reordered_indices]

K_inv = np.linalg.inv(K_approximation)
plotting.plot_matrix(K_inv, labels = reordered_labels, colorbar=True, vmax=6, vmin=-6) # vmax=1, vmin=-0.8
coords = atlas.region_coords
plotting.plot_connectome(K_inv, coords, edge_threshold = "80%", colorbar=True, edge_vmin=vmin, edge_vmax=vmax)


# Load the MSDL atlas for labels and coordinates
atlas = datasets.fetch_atlas_msdl()
atlas_labels = atlas.labels
coords = atlas.region_coords

lobe_mapping = {
    'Frontal': [8, 9, 15, 10, 16, 29, 22, 23, 31, 6, 5, 28],
    'Temporal': [0, 1, 3, 4, 25, 26, 12],
    'Parietal': [27, 30, 11, 14, 18, 17, 37, 38],
    'Occipital': [2, 7, 19, 20, 21],
    'Subcortical': [13, 24, 32, 33, 35, 34, 36]
}

# Function to create block-diagonal matrix from submatrices
def create_block_diagonal_matrix(submatrices, lobe_mapping):
    # Calculate the total number of ROIs
    num_rois = sum(len(roi_list) for roi_list in lobe_mapping.values())
    
    # Initialize the block-diagonal matrix
    block_diagonal_matrix = np.zeros((num_rois, num_rois))
    
    # Fill the block-diagonal matrix with the submatrices
    row_idx = 0
    for lobe, roi_list in lobe_mapping.items():
        size = len(roi_list)
        block_diagonal_matrix[row_idx:row_idx+size, row_idx:row_idx+size] = submatrices[lobe]
        row_idx += size
    
    return block_diagonal_matrix

# Compute the inverse of the matrix1
inverse_matrix1 = np.linalg.inv(matrix1)

# Compute the submatrices for each lobe
submatrices = {}
for lobe, roi_list in lobe_mapping.items():
    size = len(roi_list)
    submatrix = np.zeros((size, size))
    for i, roi1 in enumerate(roi_list):
        for j, roi2 in enumerate(roi_list):
            submatrix[i, j] = K_inv[roi1, roi2]
    submatrices[lobe] = submatrix

# Create the block-diagonal matrix
block_diagonal_matrix = create_block_diagonal_matrix(submatrices, lobe_mapping)

# Plot heatmap for the block-diagonal matrix
title = "Inverse K-operator as a block-diagonal matrix"
plotting.plot_matrix(block_diagonal_matrix, labels = reordered_labels, colorbar=True, vmax=6, vmin=-6, title=title)


print("Size of the block-diagonal matrix:", block_diagonal_matrix.shape)


In [ ]:

# Blocks: submatrices corresponding to the lobes

import numpy as np
from nilearn import plotting, datasets

# Load the MSDL atlas for labels and coordinates
atlas = datasets.fetch_atlas_msdl()
atlas_labels = atlas.labels
coords = atlas.region_coords

# Define a custom mapping to group ROIs by brain lobes
lobe_mapping = {
    'Frontal': [8, 9, 15, 10, 16, 29, 22, 23, 31, 6, 5, 28],
    'Temporal': [0, 1, 3, 4, 25, 26, 12],
    'Parietal': [27, 30, 11, 14, 18, 17, 37, 38],
    'Occipital': [2, 7, 19, 20, 21],
    'Subcortical etc.': [13, 24, 32, 33, 35, 34, 36]
}


# Load matrices computed for the control and PD patients
# Replace these with your actual data
matrix1 = corr_matrices[0]
matrix2 = corr_matrices_fu[0]

# Compute the inverse of the matrix1
inverse_matrix1 = np.linalg.inv(matrix1)

# Function to visualize submatrix for a given lobe
def visualize_submatrix_for_lobe(lobe):
    lobe_indices = lobe_mapping[lobe]
    submatrix = np.zeros((len(lobe_indices), len(lobe_indices)))
    submatrix_labels = [atlas_labels[i] for i in lobe_indices]  # Get ROI labels

    
    for i, roi1 in enumerate(lobe_indices):
        for j, roi2 in enumerate(lobe_indices):
            submatrix[i, j] = matrix2[roi1, roi2] * inverse_matrix1[roi2, roi1]
    
    # Plot heatmap for the submatrix
    title = f"Heatmap for Submatrix of {lobe}"
    plotting.plot_matrix(submatrix, labels=submatrix_labels, colorbar=True, vmax=1, vmin=-1, title=title)
    plotting.show()

# Visualize submatrices for each lobe
for lobe in lobe_mapping:
    visualize_submatrix_for_lobe(lobe)


In [ ]:
# SAVE the information here:


baseline_jung = corr_matrices[0]

fu_jung = corr_matrices_fu[0]

K_female_joung = K_approximation


In [ ]:
# March 24


# baseline of a new patient

# 100007, PD, M, 67, BL, fMRI

from nilearn import datasets, input_data, connectome
import nibabel as nib
import matplotlib.pyplot as plt
import numpy as np


# Load the MSDL atlas DIRECTLY HERE
atlas = datasets.fetch_atlas_msdl()
atlas_filename = atlas.maps

import os
import dicom2nifti
import dicom2nifti.settings as settings
import pydicom

# NEW DATA, from 100007


dicom_folder = "/Users/mariamannone/Downloads/other_fu/100007/rsfMRI_RL/2020-12-09_13_53_07.0/I1525726/"
output_folder = "/Users/mariamannone/Desktop/new_phd_/DICOM_converted_other_baseline/"

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Convert DICOM to NIfTI using dicom2nifti
settings.disable_validate_slice_increment()
dicom_files = [os.path.join(dicom_folder, file) for file in os.listdir(dicom_folder) if file.endswith(".dcm")]

# Load the DICOM data
dicom_data = [pydicom.dcmread(file) for file in dicom_files]

# Convert DICOM to NIfTI
nifti_file = os.path.join(output_folder, "output.nii.gz")
dicom2nifti.convert_directory(dicom_folder, nifti_file, compression=True, reorient=True)

In [ ]:
from nilearn import datasets, input_data
import os
import numpy as np
from nilearn.input_data import NiftiMapsMasker



nifti_folder = "/Users/mariamannone/Desktop/new_phd_/DICOM_converted_other_baseline/output.nii.gz/"

# Load the MSDL atlas
atlas = datasets.fetch_atlas_msdl()
atlas_filename = atlas.maps

# Create a masker using NiftiMapsMasker
masker = NiftiMapsMasker(maps_img=atlas_filename, standardize=True, memory='nilearn_cache')

# Initialize lists to store time series and labels
time_series = []
labels = []

# Iterate over each NIfTI file in the folder
for file_name in os.listdir(nifti_folder):
    if file_name.endswith('.nii.gz'):
        file_path = os.path.join(nifti_folder, file_name)

        # Load and standardize time series using the masker
        ts = masker.fit_transform(file_path)
        time_series.append(ts)

        # Extract the label from the file name or metadata
        # You may need to adapt this based on your file naming or metadata conventions
        labels.append(file_name.split('_')[1])  # Example: Assumes file_name is like 'subject_label.nii.gz'

# Save time series
for i, ts in enumerate(time_series):
    np.savetxt(f'time_series_{i:04d}.csv', ts, delimiter=',')



corr_measure = ConnectivityMeasure(kind='correlation')
pcorr_measure = ConnectivityMeasure(kind='partial correlation')

corr_matrices = corr_measure.fit_transform(time_series)
pcorr_matrices = pcorr_measure.fit_transform(time_series)



# Partial correlation matrix
plotting.plot_matrix(corr_matrices[0], labels=atlas_labels, colorbar=True, vmax=1, vmin=-0.8)

# Connectome over brain schematic
coords = atlas.region_coords # Get center coordinates of each ROI
plotting.plot_connectome(corr_matrices[0], coords, edge_threshold="80%", colorbar=True)

In [ ]:

# March 24 1st follow-up of a new patient: # 100007, PD, M, 67, V04, fMRI



# FIRST FOLLOW-UP other
# Female_fu other



import os
import dicom2nifti
import dicom2nifti.settings as settings
import pydicom

dicom_folder = "/Users/mariamannone/Downloads/other_fu/100007/rsfMRI_RL/2022-01-19_09_47_38.0/I1537553"
output_folder = "/Users/mariamannone/Desktop/new_phd_/DICOM_converted_other_fu/"

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Convert DICOM to NIfTI using dicom2nifti
settings.disable_validate_slice_increment()
dicom_files = [os.path.join(dicom_folder, file) for file in os.listdir(dicom_folder) if file.endswith(".dcm")]

# Load the DICOM data
dicom_data = [pydicom.dcmread(file) for file in dicom_files]

# Convert DICOM to NIfTI
nifti_file = os.path.join(output_folder, "output_fu.nii.gz")
dicom2nifti.convert_directory(dicom_folder, nifti_file, compression=True, reorient=True)




nifti_folder_fu = "/Users/mariamannone/Desktop/new_phd_/DICOM_converted_other_fu/output_fu.nii.gz/"



# Load the MSDL atlas
atlas = datasets.fetch_atlas_msdl()
atlas_filename = atlas.maps

# Create a masker using NiftiMapsMasker
masker = NiftiMapsMasker(maps_img=atlas_filename, standardize=True, memory='nilearn_cache')

# Initialize lists to store time series and labels
time_series_fu = []
labels = []

# Iterate over each NIfTI file in the folder
for file_name in os.listdir(nifti_folder_fu):
    if file_name.endswith('.nii.gz'):
        file_path = os.path.join(nifti_folder_fu, file_name)

        # Load and standardize time series using the masker
        ts = masker.fit_transform(file_path)
        time_series_fu.append(ts)

        # Extract the label from the file name or metadata
        # You may need to adapt this based on your file naming or metadata conventions
        labels.append(file_name.split('_')[1])  # Example: Assumes file_name is like 'subject_label.nii.gz'

# Save time series
for i, ts in enumerate(time_series_fu):
    np.savetxt(f'time_series_fu{i:04d}.csv', ts, delimiter=',')








from nilearn import datasets, input_data
import os
import numpy as np
from nilearn.input_data import NiftiMapsMasker



nifti_folder = "/Users/mariamannone/Desktop/new_phd_/DICOM_converted_other_fu/output_fu.nii.gz/"

# Load the MSDL atlas
atlas = datasets.fetch_atlas_msdl()
atlas_filename = atlas.maps

# Create a masker using NiftiMapsMasker
masker = NiftiMapsMasker(maps_img=atlas_filename, standardize=True, memory='nilearn_cache')

# Initialize lists to store time series and labels
time_series_fu = []
labels = []

# Iterate over each NIfTI file in the folder
for file_name in os.listdir(nifti_folder):
    if file_name.endswith('.nii.gz'):
        file_path = os.path.join(nifti_folder, file_name)

        # Load and standardize time series using the masker
        ts = masker.fit_transform(file_path)
        time_series_fu.append(ts)

        # Extract the label from the file name or metadata
        # You may need to adapt this based on your file naming or metadata conventions
        labels.append(file_name.split('_')[1])  # Example: Assumes file_name is like 'subject_label.nii.gz'

# Save time series
for i, ts in enumerate(time_series_fu):
    np.savetxt(f'time_series_fu{i:04d}.csv', ts, delimiter=',')





# Partial correlation matrix
plotting.plot_matrix(corr_matrices_fu[0], labels=atlas_labels, colorbar=True, vmax=1, vmin=-0.8)

# Connectome over brain schematic
coords = atlas.region_coords # Get center coordinates of each ROI
plotting.plot_connectome(corr_matrices_fu[0], coords, edge_threshold="80%", colorbar=True)








In [ ]:
# Load matrix computed above for the BASELINE
matrix1 = corr_matrices[0]

# Load matrix computed above for FIRST FOLLOW-UP
matrix2 = corr_matrices_fu[0]

inverse_matrix1 = np.linalg.inv(matrix1)

K_approximation = np.dot(matrix2,inverse_matrix1)
plotting.plot_matrix(K_approximation, labels = atlas_labels, colorbar=True, vmax=6, vmin=-6)
coords = atlas.region_coords
plt.show()


In [ ]:
# SAVE the info


baseline_alt_neu = corr_matrices[0]

fu_alt_neu = corr_matrices_fu[0]

K_male_alt_neu = K_approximation



In [ ]:
# Compute the real K-operator for the new patient
K_3_real = np.dot(fu_alt_neu, np.linalg.inv(baseline_alt_neu))
title = f"Real K-operator for the new patient"
plotting.plot_matrix(K_3_real, labels = atlas_labels, colorbar=True, vmax=6, vmin=-6)
coords = atlas.region_coords
plt.show()
